# Installing necessary packages, can ask the users to restart the notebook

In [1]:
# !sudo apt-get install -y libjpeg-dev zlib1g-dev # Do this, if pillow fails
%pip install scikit-image PyMuPDF python-docx opencv-python scipy torch torchvision==0.2.1 pillow==8.3.1 tensorflow==2.15 gdown

INFO: pip is looking at multiple versions of scikit-image to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of imageio to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of imageio to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ----------------------------- ---------- 2.4/3.2 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 3.2/3.2 MB 9.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/300.8 MB ? eta -:--:--
   -------------------

Reason for being yanked: So that users won't accidentally install this when using python 3.11


You will be prompted to Restart the session. Continue running the cells after restarting

# Importing the necessary modules

In [2]:
# Data Pre-processing
import fitz
import cv2
import csv
import os
from docx import Document
import string
import requests
import pandas as pd
import numpy as np
from PIL import Image, ImageOps
from IPython.display import clear_output as cls

# Data
from glob import glob
from tqdm import tqdm
# import tensorflow.data as tfd

# Data Visualization
import matplotlib.pyplot as plt

# Model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from tensorflow.keras.utils import plot_model

# Downloading the dataset

In [2]:
# Downloading the Dataset on collab/local notebook

# Virtuosa ( Dataset 1)
!gdown 'http://drive.google.com/uc?id=10NX_UbV2HMbPEO2fvKYAIXOOOec0g38g'  
# Downloading link for Ancient Text
!gdown 'http://drive.google.com/uc?id=1YTaqNoZCYP74AuQxlyJsiQLhcoc8DNSv'  
# Downloading link for Ground Truth Text

# Perfecto ( Dataset 2)
!gdown 'http://drive.google.com/uc?id=1x6FS3z4WhsHS7s38a2oSH8JnLQ_u_f21'  
# Downloading link for Ancient Text
!gdown 'http://drive.google.com/uc?id=1YqQ04ZQR4xdjKlQS9xL9zkkWYx_Ppxwa'  
# Downloading link for Ground Truth Text

!gdown 'http://drive.google.com/uc?id=1r7TjJ9RjNZHxAzKhd4uOaRWanrQXIqw8' 
# Downloading utils.py from cloud

Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id='http://drive.google.com/uc?id=10NX_UbV2HMbPEO2fvKYAIXOOOec0g38g'

but Gdown can't. Please check connections and permissions.
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id='http://drive.google.com/uc?id=1YTaqNoZCYP74AuQxlyJsiQLhcoc8DNSv'

but Gdown can't. Please check connections and permissions.
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You 

In [9]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


# Pre-processing the dataset

### Converting PDF to images

In [37]:
from utils import pdf_to_images

pdf_path1 = "./Padilla - Nobleza virtuosa_testExtract.pdf"  # Path to the PDF file
unproc_images_folder_1 = "./preprocessing/imgsUnProcessed1"  # Output folder to save the images
if not os.path.exists(unproc_images_folder_1):
    os.makedirs(unproc_images_folder_1)
pdf_to_images(pdf_path1, unproc_images_folder_1)



<img src="https://raw.githubusercontent.com/ML4SCI/DeepLearnHackathon/main/NLPRenaissanceChallenge/images/imageOriginal.png">

### Splitting two sided scanned images into individual pages.

In [38]:
#Segregating 2-sided pages into individual pages
from utils import process_images

unproc_images_folder_1 = "./preprocessing/imgsUnProcessed1"
proc_images_folder_1 = "./preprocessing/imgsForAllPages1"
if not os.path.exists(proc_images_folder_1):
    os.makedirs(proc_images_folder_1)
process_images(unproc_images_folder_1, proc_images_folder_1)


print("Image processing complete!")

Image processing complete!


In [10]:
!git clone https://github.com/Shashankss1205/CRAFT_Model

Cloning into 'CRAFT_Model'...


# Text Detection
### Extracting words from a scanned text page image can be achieved using any model of your choice. We are using the [CRAFT Model](https://github.com/clovaai/CRAFT-pytorch) for the same. (This will take 1-2 mins to process the entire model)

In [11]:
#It generally takes about ~3-4 mins
!python CRAFT_Model/CRAFT/BoundBoxFunc/test.py --result_folder="./preprocessing/BoundBoxApplied2/" --test_folder="./preprocessing/imgsForAllPages1" --trained_model="CRAFT_Model/CRAFT/BoundBoxFunc/weights/craft_mlt_25k.pth"

### The output of this model provides coordinates of the polygon enclosing the word. Using these coordinates one can draw a bounding box and crop word images.

In [33]:
from utils import process_bounding_boxes

#Sorting the BB based on the Spanish writing style
def sort_bounding_boxes(input_directory, output_directory):
    # Iterate over each text file in the directory
    for filename in os.listdir(input_directory):
        if filename.endswith(".txt"):
            file_path = os.path.join(input_directory, filename)
            sorted_bounding_boxes = process_bounding_boxes(file_path)

            # Write sorted bounding boxes to text file in output directory
            output_file_path = os.path.join(output_directory, f"{os.path.splitext(filename)[0]}_sorted.txt")
            with open(output_file_path, "w") as outfile:
                for group in sorted_bounding_boxes:
                    for box in group:
                        outfile.write(','.join(map(str, box)) + '\n')
                    # Optionally, write a separator between groups, e.g., a blank line:
                    outfile.write('\n')


bound_box_applied1 = './preprocessing/BoundBoxApplied1/'
bound_box_sorted1 = "./preprocessing/BoundBoxSorted1"
if not os.path.exists(bound_box_sorted1):
    os.makedirs(bound_box_sorted1)
sort_bounding_boxes(bound_box_applied1, bound_box_sorted1)

bound_box_applied2 = './preprocessing/BoundBoxApplied2/'
bound_box_sorted2 = "./preprocessing/BoundBoxSorted2"
if not os.path.exists(bound_box_sorted2):
    os.makedirs(bound_box_sorted2)
sort_bounding_boxes(bound_box_applied2, bound_box_sorted2)

ValueError: invalid literal for int() with base 10: ''

<img src="https://raw.githubusercontent.com/ML4SCI/DeepLearnHackathon/main/NLPRenaissanceChallenge/images/imageCRAFT.jpg">

### Extracting all the words from .docx file containing Transcription (True lables).

In [ ]:
#ground truth to text code
from utils import save_pages_to_text

docx_file1 = "./Padilla - 1 Nobleza virtuosa_testTranscription.docx"
grnd_truth1 = "./preprocessing/all_text1.txt"     # File where the complete processed text will be saved
save_pages_to_text(docx_file1, grnd_truth1)

docx_file2 = "./Padilla - 2 Noble perfecto_Transcription.docx"
grnd_truth2 = "./preprocessing/all_text2.txt"     # File where the complete processed text will be saved
save_pages_to_text(docx_file2, grnd_truth2)

### Since we have processed our text book into individual pages, we need to split the entire transcription based on text pages.

In [ ]:
#splitting text files, based on the text
from utils import process_textfiles

grnd_truth1 = "./preprocessing/all_text1.txt"
bound_box_sorted1 = './preprocessing/BoundBoxSorted1'
proc_grnd_truth1 = "./preprocessing/textSplitted1"
TEST_SIZE=6
if not os.path.exists(proc_grnd_truth1):
    os.makedirs(proc_grnd_truth1)
process_textfiles(grnd_truth1, bound_box_sorted1, proc_grnd_truth1, TEST_SIZE)


grnd_truth2 = "./preprocessing/all_text2.txt" 
bound_box_sorted2 = './preprocessing/BoundBoxSorted2'
proc_grnd_truth2 = "./preprocessing/textSplitted2"
TEST_SIZE=0
if not os.path.exists(proc_grnd_truth2):
    os.makedirs(proc_grnd_truth2)
process_textfiles(grnd_truth2, bound_box_sorted2, proc_grnd_truth2, TEST_SIZE)
print("Text splitting complete!")

31 files in the folder
31 files in the folder
Text splitting complete!


### Extracts and saves bounding boxes from images using text data for filenames, skipping the last 6 images for testing.

In [ ]:
from utils import apply_extraction_to_folder_for_train, count_files_in_folder

proc_images_folder_1 = './preprocessing/imgsForAllPages1'
bound_box_sorted1 = './preprocessing/BoundBoxSorted1'
proc_grnd_truth1 = './preprocessing/textSplitted1'
training_data1 = './traning_data1'
test_size=6
train_size = count_files_in_folder(proc_images_folder_1, ['.png', '.jpeg', '.jpg'])- test_size
print("Training pages " +  str(train_size))
if not os.path.exists(training_data1):
    os.makedirs(training_data1)
apply_extraction_to_folder_for_train(proc_images_folder_1, bound_box_sorted1, proc_grnd_truth1, training_data1, train_size)


proc_images_folder_2 = './preprocessing/imgsForAllPages2'
bound_box_sorted2 = './preprocessing/BoundBoxSorted2'
proc_grnd_truth2 = './preprocessing/textSplitted2'
training_data2 = './traning_data2'
test_size = 0
train_size = count_files_in_folder(proc_images_folder_1, ['.png', '.jpeg', '.jpg'])- test_size
print("Training pages " +  str(train_size))
if not os.path.exists(training_data2):
    os.makedirs(training_data2)
apply_extraction_to_folder_for_train(proc_images_folder_2, bound_box_sorted2, proc_grnd_truth2, training_data2, train_size) # better to send no. of pages given in transcription

Training pages 25


Training pages 31
